In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
import scipy.stats as stats

file = 'datasets/atletas.csv'
df = pd.read_csv(file)
print(df)

            Atleta  Altura   Peso    Sexo  Ano de nascimento  tempo  _tempo
0          Adriano   190.0  120.0   homem             1987.0     22      25
1        Alexandre   170.0   62.0   homem                NaN     15      12
2          Antonio   180.0   78.0   homem             2002.0     16      15
3           Arthur   164.0   66.0   homem                NaN     14      17
4     Arthur Costa   173.0   76.0   homem             2005.0     24      16
5           Caique   180.0   60.0   homem             2002.0     11      14
6         Caroline   180.0   65.0  mulher             1997.0     20      16
7            Diogo   195.0   85.0   homem             1996.0     10      20
8          Eduardo   180.0   75.0   homem                NaN     16      19
9          Fabiano   174.0   94.0   homem             2002.0     10      23
10  Giovanna Pires   166.0   55.0  mulher             2002.0     12      14
11       Jaqueline   170.0   77.0  mulher             1986.0     19      19
12          

In [14]:
df1 = df
df1['dummy_gender'] = df.apply(lambda row: 1 if row['Sexo'] == 'homem' else 0, axis=1)
df_filtered = df1.query('Altura.notnull() and Peso.notnull() and `Ano de nascimento`.notnull()')
df_filtered


,Atleta,Altura,Peso,Sexo,Ano de nascimento,tempo,_tempo,dummy_gender
0,Adriano,190.0,120.0,homem,1987.0,22,25,1
2,Antonio,180.0,78.0,homem,2002.0,16,15,1
4,Arthur Costa,173.0,76.0,homem,2005.0,24,16,1
5,Caique,180.0,60.0,homem,2002.0,11,14,1
6,Caroline,180.0,65.0,mulher,1997.0,20,16,0
7,Diogo,195.0,85.0,homem,1996.0,10,20,1
9,Fabiano,174.0,94.0,homem,2002.0,10,23,1
10,Giovanna Pires,166.0,55.0,mulher,2002.0,12,14,0
11,Jaqueline,170.0,77.0,mulher,1986.0,19,19,0
12,John,180.0,87.0,homem,2001.0,20,21,1


In [15]:
x = df_filtered[['dummy_gender', 'Ano de nascimento', 'Peso', 'Altura']]
y = df_filtered['tempo'].to_numpy()
model = LinearRegression()
model.fit(x, y)
score = model.score(x, y)
print(score)

0.28165797485702493


In [16]:
# Vou começar a reconstruir o ano de nascimento
# Vou utilizar como variáveis regressoras o peso, altura e sexo

df_filtered = df1[ (df1['Peso'].notnull()) & (df1['Altura'].notnull()) & (df1['Ano de nascimento'].notnull()) ]
x = df_filtered[['dummy_gender', 'Altura', 'Peso']].to_numpy()
y = df_filtered['Ano de nascimento'].to_numpy()
model_ano = LinearRegression()
model_ano.fit(x, y)
score = model_ano.score(x, y)
score

0.6446571873527764

In [17]:
# minha função criada para completar o ano de cada linha
def completar_ano(linha):
  # verifica se o ano de nascimento NÃO está definido e se Peso e Altura estão
  # Só calcula o ano de nascimento se o não existir ano de nascimento
  # Só calcula o ano de nascimento se tiver Peso e Altura para fornecer para o modelo
  if pd.isna(linha['Ano de nascimento']) and not pd.isna(linha['Peso']) and not pd.isna(linha['Altura']):
    # Separa as variáveis regressoras utilizadas no modelo
    campos = linha[['dummy_gender','Altura','Peso']]
    # Retorna para o "apply" o valor previsto
    return round(model_ano.predict([campos])[0])
  # Se já tem o ano de nascimento, devolve o valor que tem
  return linha['Ano de nascimento']

# apply: executar a função chamada 1x para cada linha
df1['ano_calculado'] = df1.apply(completar_ano, axis=1)
df1


,Atleta,Altura,Peso,Sexo,Ano de nascimento,tempo,_tempo,dummy_gender,ano_calculado
0,Adriano,190.0,120.0,homem,1987.0,22,25,1,1987.0
1,Alexandre,170.0,62.0,homem,NaN,15,12,1,2010.0
2,Antonio,180.0,78.0,homem,2002.0,16,15,1,2002.0
3,Arthur,164.0,66.0,homem,NaN,14,17,1,2009.0
4,Arthur Costa,173.0,76.0,homem,2005.0,24,16,1,2005.0
5,Caique,180.0,60.0,homem,2002.0,11,14,1,2002.0
6,Caroline,180.0,65.0,mulher,1997.0,20,16,0,1997.0
7,Diogo,195.0,85.0,homem,1996.0,10,20,1,1996.0
8,Eduardo,180.0,75.0,homem,NaN,16,19,1,2004.0
9,Fabiano,174.0,94.0,homem,2002.0,10,23,1,2002.0


In [18]:
df_filtered = df1[ (df1['Peso'].notnull()) & (df1['Altura'].notnull()) & (df1['ano_calculado'].notnull()) ]
x = df_filtered[['dummy_gender', 'ano_calculado']].to_numpy()
y = df_filtered['Peso'].to_numpy()
model_peso = LinearRegression()
model_peso.fit(x, y)
score = model_peso.score(x, y)
score
# gerei modelo de previsão para o peso utilizando como variáveis regressoras o sexo e o ano de nascimento calculado


0.7191879283286167

In [19]:
df_filtered = df1[ (df1['Peso'].notnull()) & (df1['Altura'].notnull()) & (df1['ano_calculado'].notnull()) ]
x = df_filtered[['dummy_gender', 'ano_calculado']].to_numpy()
y = df_filtered['Altura'].to_numpy()
model_altura = LinearRegression()
model_altura.fit(x, y)
score = model_altura.score(x, y)
score
# gerei modelo de previsão para o peso utilizando como variáveis regressoras o sexo e o ano de nascimento calculado


0.4936899707102512

In [20]:
# minha função criada para completar o peso de cada linha
def completar_peso(linha):
  if pd.isna(linha['Peso']) and not pd.isna(linha['ano_calculado']):
    campos = linha[['dummy_gender','ano_calculado']]
    return round(model_peso.predict([campos])[0])
  return linha['Peso']

# apply: executar a função chamada 1x para cada linha
df1['peso_calculado'] = df1.apply(completar_peso, axis=1)
df1

,Atleta,Altura,Peso,Sexo,Ano de nascimento,tempo,_tempo,dummy_gender,ano_calculado,peso_calculado
0,Adriano,190.0,120.0,homem,1987.0,22,25,1,1987.0,120.0
1,Alexandre,170.0,62.0,homem,NaN,15,12,1,2010.0,62.0
2,Antonio,180.0,78.0,homem,2002.0,16,15,1,2002.0,78.0
3,Arthur,164.0,66.0,homem,NaN,14,17,1,2009.0,66.0
4,Arthur Costa,173.0,76.0,homem,2005.0,24,16,1,2005.0,76.0
5,Caique,180.0,60.0,homem,2002.0,11,14,1,2002.0,60.0
6,Caroline,180.0,65.0,mulher,1997.0,20,16,0,1997.0,65.0
7,Diogo,195.0,85.0,homem,1996.0,10,20,1,1996.0,85.0
8,Eduardo,180.0,75.0,homem,NaN,16,19,1,2004.0,75.0
9,Fabiano,174.0,94.0,homem,2002.0,10,23,1,2002.0,94.0


In [21]:
# minha função criada para completar o peso de cada linha
def completar_altura(linha):
  if pd.isna(linha['Altura']) and not pd.isna(linha['ano_calculado']):
    campos = linha[['dummy_gender','ano_calculado']]
    return round(model_altura.predict([campos])[0])
  return linha['Altura']

# apply: executar a função chamada 1x para cada linha
df1['altura_calculada'] = df1.apply(completar_altura, axis=1)
df1

,Atleta,Altura,Peso,Sexo,Ano de nascimento,tempo,_tempo,dummy_gender,ano_calculado,peso_calculado,altura_calculada
0,Adriano,190.0,120.0,homem,1987.0,22,25,1,1987.0,120.0,190.0
1,Alexandre,170.0,62.0,homem,NaN,15,12,1,2010.0,62.0,170.0
2,Antonio,180.0,78.0,homem,2002.0,16,15,1,2002.0,78.0,180.0
3,Arthur,164.0,66.0,homem,NaN,14,17,1,2009.0,66.0,164.0
4,Arthur Costa,173.0,76.0,homem,2005.0,24,16,1,2005.0,76.0,173.0
5,Caique,180.0,60.0,homem,2002.0,11,14,1,2002.0,60.0,180.0
6,Caroline,180.0,65.0,mulher,1997.0,20,16,0,1997.0,65.0,180.0
7,Diogo,195.0,85.0,homem,1996.0,10,20,1,1996.0,85.0,195.0
8,Eduardo,180.0,75.0,homem,NaN,16,19,1,2004.0,75.0,180.0
9,Fabiano,174.0,94.0,homem,2002.0,10,23,1,2002.0,94.0,174.0


In [22]:
# Recalculo a minha regressão original utilizando as variáveis novas
x = df1[['dummy_gender', 'ano_calculado', 'peso_calculado', 'altura_calculada']]
y = df1['tempo'].to_numpy()
model = LinearRegression()
model.fit(x, y)
score = model.score(x, y)
print(score)

0.3110657932399812
